In [1]:
from pandas import read_excel
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Helper fuctions

In [2]:
def load_csv(filename):
    file = open(filename, "r", encoding="utf-8")
    experiments = []
    headers = []
    reader = csv.reader(file)
    for i, lines in enumerate(reader):
        if i == 0:
            #headers = [e for e in lines[0].split(";")]
            h = lines[0].split(";")
            headers.append(h)
        else:
            tmp = lines[0].split(";")
            for j in range(0,len(tmp)):
                if (j != 0 and j != 1879):
                    tmp[j] = float(tmp[j])
                else:
                    continue
            #tmp = [float(tmp[j]) for j in range(0,len(tmp)) if (j!=0 and j!=1879)] #This removes two entries, dunno why. 
            experiments.append(tmp)
    file.close()
    return headers, experiments

def column_extraction(data_list,column_number): #columns start counting in 0
    col = []
    for i in range(0,len(experiments)):
        dato = experiments[i][column_number]
        col.append(dato)
    return col   

## Load both data sets for comparision

In [3]:
# Loading compound library
my_sheet = 'SPECS_drug_repurp_set' # change it to your sheet name, you can find your sheet name at the bottom left of your excel file
file_name = 'SPECS_drug_repurp_set.xlsx' # change it to the name of your excel file
df = read_excel(file_name, sheet_name = my_sheet)
#print(df.head()) # shows headers with top 5 rows
library_compound_ids = df[["Compound ID"]].copy()
library_compound_ids.rename(columns = {'Compound ID':'ID library'}, inplace = True)

In [4]:
#Loading Maris' data
headers, experiments =load_csv('/home/jovyan/covid-data/dalia_MeanFeaturesPerWell_BatchA.csv')
#Column 1879 contains the compound ID
screening_compound_ids = pd.DataFrame(column_extraction(experiments,1879), columns=['ID Screening'])

In [5]:
print(f'The library has {len(library_compound_ids)} compounds')
print(f'The experiment has {len(screening_compound_ids)} compounds')

The library has 5280 compounds
The experiment has 6065 compounds


### Cleaning Library's compounds

In [6]:
# Removing compunds with ID "Problem with structure"
problem = library_compound_ids.loc[library_compound_ids['ID library']=='Problem with structure']
problem

,ID library
3471,Problem with structure
5185,Problem with structure


In [7]:
#Check that we did not miss information
library_compound_ids_2 = library_compound_ids.copy().drop([3471, 5185])
print(f'The library has now {len(library_compound_ids_2)} compounds')

The library has now 5278 compounds


In [8]:
#Check uniqueness in column "Compound ID"
aux1 = library_compound_ids_2['ID library'].copy().unique() #Array with all unique ID's names
print(f'There are {len(library_compound_ids_2) - len(aux1)} non unique compounds\n')
aux2 = library_compound_ids_2.copy().groupby('ID library').size().sort_values(ascending=False).to_frame() #Count the elements in each group and sort
aux2.rename(columns = {0:'Count'}, inplace = True)
print(f'Printing the list of grouped compunds\n {aux2}')

There are 90 non unique compounds

Printing the list of grouped compunds
             Count
ID library       
CBK011558       3
CBK011603       3
CBK042067       3
CBK041250       3
CBK000879       3
...           ...
CBK278033       1
CBK278032       1
CBK278031       1
CBK278030       1
CBK309769G      1

[5188 rows x 1 columns]


### Cleaning experiment's compounds

In [9]:
#Check uniqueness in column "Compound ID"
aux11 = screening_compound_ids['ID Screening'].copy().unique() #Array with all unique ID's names
print(f'There are {len(screening_compound_ids) - len(aux11)} non unique compounds\n')
aux21 = screening_compound_ids.copy().groupby('ID Screening').size().sort_values(ascending=False).to_frame() #Count the elements in each group and sort
aux21.rename(columns = {0:'Count'}, inplace = True)
print(f'Printing the list of grouped compunds\n {aux21}')

There are 881 non unique compounds

Printing the list of grouped compunds
               Count
ID Screening       
Uninfected      256
DMSO            188
Remdesivir      158
CP4              48
CP3              48
...             ...
CBK278048         1
CBK278047         1
CBK278045         1
CBK278044         1
CBK290655         1

[5184 rows x 1 columns]


In [ ]:
print(f'Printing a part of the list of grouped compunds in the SPEC library\n {aux2[0:15]}')
print(f'Printing a part of the list of grouped compunds in the experiment\n {aux21[0:15]}')